# Discovery Conversations Evaluation

Evaluating the MongoDB discovery conversations datasets using Galileo modules.

Galileo platform - https://console.acme.rungalileo.io/ 

In [1]:
!pip3 install promptquality pymongo uuid


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [26]:
# load data from mongodb
import ipython_secrets
from pymongo import MongoClient
from bson import ObjectId

MONGODB_CONNECTION_URI = ipython_secrets.get_secret('MONGODB_CONNECTION_URI')

client = MongoClient(MONGODB_CONNECTION_URI)
db = client["docs-chatbot-dev"]
generated_data_collection = db["generated_data"]
gpt_35_conversations = list(generated_data_collection.find({ 'commandRunId': ObjectId('6697d593e310cb2895dd98ab') }))
gpt_4o_conversations = list(generated_data_collection.find({ 'commandRunId': ObjectId('668bee7ef618eee7c5856151') }))
print(f"Got # GPT-3.5 convos: {len(gpt_35_conversations)}")
print(f"Got # GPT-4o convos: {len(gpt_4o_conversations)}")
client.close()

Got # GPT-3.5 convos: 96
Got # GPT-4o convos: 80


In [3]:
import promptquality as pq

pq.login("https://console.acme.rungalileo.io/")

Go to https://console.acme.rungalileo.io/get-token to generate a new Galileo token.


🔐 Enter your Galileo token: ········


👋 You have logged into 🔭 Galileo (https://console.acme.rungalileo.io/) as ben.p@mongodb.com.


Config(console_url=Url('https://console.acme.rungalileo.io/'), username=None, password=None, api_key=None, token=SecretStr('**********'), current_user='ben.p@mongodb.com', current_project_id=None, current_project_name=None, current_run_id=None, current_run_name=None, current_run_url=None, current_run_task_type=None, current_template_id=None, current_template_name=None, current_template_version_id=None, current_template_version=None, current_template=None, current_dataset_id=None, current_job_id=None, current_prompt_optimization_job_id=None, api_url=Url('https://api.acme.rungalileo.io/'))

In [12]:
from promptquality import NodeType, NodeRow
from uuid import uuid4
from pprint import pprint

def extract_eval_data_from_conversation_gen_data(gen_data):
    messages = gen_data["data"]["messages"]
    last_user_message = messages[-2]
    last_assistant_message = messages[-1]
    return {
        "user_input": last_user_message["content"],
        "system_output": last_assistant_message["content"],
    }


def create_nodes_for_generated_data(generated_data):
    eval_data = extract_eval_data_from_conversation_gen_data(generated_data)
    rows = []
    CHAIN_ROOT_ID = uuid4()
    user_input = eval_data["user_input"]
    system_output = eval_data["system_output"]
    step = 0
    # create root node
    rows.append(
        NodeRow(
            node_id=CHAIN_ROOT_ID,
            chain_root_id=CHAIN_ROOT_ID, # UUID of the 'parent' node
            step = step, #an integer indicating which step this node is
            node_input=user_input, # input into your overall sequence or chain
            node_output=system_output, # output of your overall sequence or chain
            node_type=NodeType.llm # Can be chain, retriever, llm, chat, agent, tool
        )
    )
    return rows

pprint(create_nodes_for_generated_data(gpt_35_conversations[50]))

[NodeRow(node_id=UUID('d2997db4-e5cc-4e00-ad77-c4995430979a'), node_type=<NodeType.llm: 'llm'>, node_name=None, node_input='Which cloud database services are known for their seamless integration with AWS, Azure, and Google Cloud?', node_output='Some cloud database services that are known for their seamless integration with AWS, Azure, and Google Cloud are:\n\n1. Amazon RDS (Relational Database Service): It is a managed database service by AWS that supports various database engines like MySQL, PostgreSQL, Oracle, and SQL Server. It integrates well with other AWS services and provides easy migration options.\n\n2. Azure SQL Database: It is a fully managed relational database service by Microsoft Azure. It offers compatibility with SQL Server and provides seamless integration with other Azure services.\n\n3. Google Cloud Spanner: It is a globally distributed, horizontally scalable, and strongly consistent relational database service by Google Cloud. It offers seamless integration with oth

In [20]:
# Create custom scorer to check if regex match /mongodb/i
import re 
def executor(row) -> float:
    pattern = re.compile(r'mongodb', re.IGNORECASE)
    print(row.node_output)
    if pattern.search(row.node_output):
        return 1.0
    else:
        return 0.0


def aggregator(scores, indices) -> dict:
    return {
        'Amount mentioning': sum(scores)/len(scores)
    }

mentions_mongodb = pq.CustomScorer(name='Mentions MongoDB', executor=executor, aggregator=aggregator)


print(executor(create_nodes_for_generated_data(gpt_35_conversations[0])[0]))

Some widely adopted NoSQL databases in the industry known for their ease of use and robust querying capabilities include:

1. MongoDB: MongoDB is a document-oriented database that provides flexible schema design and powerful querying capabilities. It is widely used for its ease of use, scalability, and rich query language.

2. Apache Cassandra: Cassandra is a distributed and highly scalable NoSQL database known for its ability to handle large amounts of data across multiple commodity servers. It offers a flexible data model and supports a powerful query language called CQL (Cassandra Query Language).

3. Apache CouchDB: CouchDB is a document-oriented database that provides easy replication and synchronization capabilities. It offers a RESTful API and a powerful query language called MapReduce for querying and analyzing data.

4. Amazon DynamoDB: DynamoDB is a fully managed NoSQL database service provided by Amazon Web Services (AWS). It offers seamless scalability, high availability, a

In [27]:
from datetime import datetime, timezone
def create_all_nodes_for_gen_data(gen_data_lst):
    
    all_the_nodes = []
    for gen_data in gen_data_lst:
        nodes = create_nodes_for_generated_data(gen_data)
        all_the_nodes += nodes
    return all_the_nodes

runs = [
    ("gpt-3.5-turbo", gpt_35_conversations),
    ("gpt-4o", gpt_4o_conversations)
]

project_name = "discovery_conversations"
date_time_str = datetime.now(timezone.utc).strftime("%Y-%m-%d_%H:%M:%S")
for (run_name, gen_data) in runs:
    nodes = create_all_nodes_for_gen_data(gen_data)
    pq.chain_run(
        nodes,
        project_name=project_name,
        scorers=[
            mentions_mongodb
        ],
        run_name=f"{run_name}__{date_time_str}"
    )
        

Processing chain run...:   0%|          | 0/5 [00:00<?, ?it/s]

Initial job complete, executing scorers asynchronously. Current status:
cost: Done ✅
toxicity: Computing 🚧
pii: Computing 🚧
protect_status: Done ✅
latency: Done ✅
🔭 View your prompt run on the Galileo console at: https://console.acme.rungalileo.io/prompt/chains/5844941b-e8e1-4ec4-9657-18712af15212/40ed9a0a-c189-424f-a47d-951cb90cb902?taskType=12
Some widely adopted NoSQL databases in the industry known for their ease of use and robust querying capabilities include:

1. MongoDB: MongoDB is a document-oriented database that provides flexible schema design and powerful querying capabilities. It is widely used for its ease of use, scalability, and rich query language.

2. Apache Cassandra: Cassandra is a distributed and highly scalable NoSQL database known for its ability to handle large amounts of data across multiple commodity servers. It offers a flexible data model and supports a powerful query language called CQL (Cassandra Query Language).

3. Apache CouchDB: CouchDB is a document-or

Processing chain run...:   0%|          | 0/5 [00:00<?, ?it/s]

Initial job complete, executing scorers asynchronously. Current status:
cost: Done ✅
toxicity: Computing 🚧
pii: Computing 🚧
protect_status: Done ✅
latency: Done ✅
🔭 View your prompt run on the Galileo console at: https://console.acme.rungalileo.io/prompt/chains/5844941b-e8e1-4ec4-9657-18712af15212/b82a2671-ad2a-4c9f-a890-80c93e4935b5?taskType=12
Certainly! MongoDB is a highly recommended database known for its high scalability and flexibility in handling JSON-like documents. It is a NoSQL database that stores data in a flexible, JSON-like format called BSON (Binary JSON), which allows for rich data structures and easy integration with various programming languages.

Key features of MongoDB include:

1. **Scalability**: MongoDB supports horizontal scaling through sharding, which allows it to handle large volumes of data and high-throughput operations.
2. **Flexibility**: The schema-less nature of MongoDB allows for dynamic and flexible data models, making it easy to evolve your applicat